**Enhancing Customer Retention through Machine Learning**: <br> Segmentation, Churn Prediction, and Product Recommendation – A Case Study on Flipkart Data.
<br><br>
Phase: **Data Preprocessing** <br>
By: Gia-My Nguyen <br>
Last updated (dd/mm/yyyy): 25/09/2025
<br><br>

**Completed:**
- Dropped redundant ('unnamed') columns & NULLs in *total_weighted_landing_price*.
- Identified "0" values in *procured_quantity*, *unit_selling_price*.
- Remove records with procured_quantity = 0, keep only those > 0."
- Checked consistency between category and category_id, handled the 'Syrups' case.
- Supporting files for easier cross-referencing (Location: data/supporting).
<br><br>

**Tasks Remaining**
- Handle "0" values in *unit_selling_price*
- Standardize str columns (before modelling)
- Create 'Revenue' 



In [1]:
import pandas as pd

# Functions

In [2]:
"""
    Convert all specified ID columns to string type.
    
    Parameters:
        df (pd.DataFrame): Input dataframe
        id_columns (list, optional): List of ID columns to convert. 
                                     If None, will auto-detect columns ending with '_id'.
    
    Returns:
        pd.DataFrame: Dataframe with converted ID columns
"""
    
def convert_id_to_str(df, id_columns=None):
    if id_columns is None:
        id_columns = [col for col in df.columns if col.endswith("_id") or col == 'dim_customer_key']
    
    for col in id_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)
    
    return df


In [3]:
"""
    Create a mapping between 'category_id' and category_name'.  
    If a category name has multiple IDs -> append the ID to the name.
"""

def build_mapping(df, id_col, name_col):
    
    # Category name with multiple IDs
    dup_names = (
        df.groupby(name_col)[id_col]
        .nunique()
        .reset_index()
    )
    dup_names = dup_names[dup_names[id_col] > 1][name_col].tolist()

    # Handle duplicated names
    def rename(row):
        if row[name_col] in dup_names:
            return f"{row[name_col]} ({row[id_col]})"
        return row[name_col]

    df[name_col] = df.apply(rename, axis=1)

    # mapping data
    mapping = (
        df[[id_col, name_col]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    return mapping


In [46]:
# def clean_double_quotes(df, col_name):
#     """
#     Clean column:
#     - Remove surrounding double quotes
#     - Remove all extra quotes inside text
#     - Strip leading/trailing spaces
#     - Normalize multiple spaces into one
#     """
#     df[col_name] = (
#         df[col_name]
#         .astype(str)                                  # đảm bảo dữ liệu là string
#         .str.strip('"')                               # bỏ " ở đầu và cuối
#         # .str.replace('"', '', regex=False)            # bỏ " còn sót bên trong
#         .str.strip()                                  # bỏ khoảng trắng đầu/cuối
#         .str.replace(r'\s+', ' ', regex=True)         # chuẩn hóa khoảng trắng
#     )
#     return df


# Sales

In [19]:
# read file

df_sales = pd.read_csv('data/raw/Sales.csv')

In [20]:
# Basic information

df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46706387 entries, 0 to 46706386
Data columns (total 13 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0.2                  int64  
 1   Unnamed: 0.1                  int64  
 2   Unnamed: 0                    int64  
 3   date_                         object 
 4   city_name                     object 
 5   order_id                      int64  
 6   cart_id                       int64  
 7   dim_customer_key              int64  
 8   procured_quantity             int64  
 9   unit_selling_price            float64
 10  total_discount_amount         float64
 11  product_id                    int64  
 12  total_weighted_landing_price  float64
dtypes: float64(3), int64(8), object(2)
memory usage: 4.5+ GB


In [21]:
# Drop "Unnamed" columns

cols_to_drop = [col for col in df_sales.columns if 'Unnamed' in col]
df_sales = df_sales.drop(columns=cols_to_drop)

In [7]:
# Check missing values

df_sales.isna().sum()

date_                               0
city_name                           0
order_id                            0
cart_id                             0
dim_customer_key                    0
procured_quantity                   0
unit_selling_price                  0
total_discount_amount               0
product_id                          0
total_weighted_landing_price    79355
dtype: int64

## Problem 01: <br> **79,355 missing values in "Total Weighted Landing Price"**

File: sales_missing_values.csv

**Proposal**

- Remove all these records from the master data

In [ ]:
# sales_missing_values.csv

# sales_missing_values = df_sales[df_sales['total_weighted_landing_price'].isnull()]
# print(sales_missing_values.shape[0])

# sales_missing_values.to_csv('data/anomalies/sales_missing_values.csv')

In [22]:
# Drop all records where total_weighted_landing_price is NULL

df_sales_cleaned = df_sales.dropna(subset=['total_weighted_landing_price'])
print("Count after dropping NULL in total_weighted_landing_price:", len(df_sales_cleaned))

Count after dropping NULL in total_weighted_landing_price: 46627032


In [9]:
# Check NULL values again

df_sales_cleaned.isna().sum()

date_                           0
city_name                       0
order_id                        0
cart_id                         0
dim_customer_key                0
procured_quantity               0
unit_selling_price              0
total_discount_amount           0
product_id                      0
total_weighted_landing_price    0
dtype: int64

In [10]:
# Check abnormal data in 'procured_quantity' & 'unit_selling_price'

cols_to_check = ['procured_quantity', 'unit_selling_price']

for col in cols_to_check:
    zero_count = (df_sales_cleaned[col] == 0).sum()
    negative_count = (df_sales_cleaned[col] < 0).sum()
    
    print(f"Column: {col}")
    print(f"  Records = 0: {zero_count}")
    print(f"  Records < 0: {negative_count}\n")


Column: procured_quantity
  Records = 0: 178908
  Records < 0: 0

Column: unit_selling_price
  Records = 0: 48269
  Records < 0: 0



## Problem 02: <br> **178,908 records with procured_quantity = 0**
File: sales_zero_quantity.csv <br> <br>
No products were purchased -> Invalid transactions <br>
Possible reason: system error

**Proposal**: drop

In [ ]:
# sales_zero_quantity.csv

# sales_zero_quantity = df_sales_cleaned[df_sales_cleaned['procured_quantity'] == 0]
# sales_zero_quantity.to_csv('data/anomalies/sales_zero_quantity.csv')

In [23]:
# Only get records with procured_quantity > 0

df_sales_cleaned = df_sales_cleaned[df_sales_cleaned['procured_quantity'] > 0]

# check agian
print("Records w. 'procured_quantity=0': ",df_sales_cleaned[df_sales_cleaned['procured_quantity'] == 0].shape[0])

Records w. 'procured_quantity=0':  0


## Problem 03: <br> **48,269 records with unit_selling_price = 0**
File: sales_zero_unit_price.csv<br> <br>
Products sold at zero price <br>
Possible reasons:
- Sample products / free gifts / special promotions
- Error in recording selling price

**Proposal**: add flag *is_free_product*

In [ ]:
# sales_zero_unit_price.csv

# sales_zero_unit_price = df_sales_cleaned[df_sales_cleaned['unit_selling_price'] == 0]
# sales_zero_unit_price.to_csv('data/anomalies/sales_zero_unit_price.csv')

In [24]:
# Rename the date column and convert to datetime objects

df_sales_cleaned.rename(columns={'date_': 'date'}, inplace=True)
df_sales_cleaned['date'] = pd.to_datetime(df_sales_cleaned['date'])

In [25]:
# Convert ids to str

df_sales_cleaned = convert_id_to_str(df_sales_cleaned)

In [14]:
df_sales_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46448124 entries, 0 to 46706386
Data columns (total 10 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   date                          datetime64[ns]
 1   city_name                     object        
 2   order_id                      object        
 3   cart_id                       object        
 4   dim_customer_key              object        
 5   procured_quantity             int64         
 6   unit_selling_price            float64       
 7   total_discount_amount         float64       
 8   product_id                    object        
 9   total_weighted_landing_price  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 3.8+ GB


In [15]:
df_sales_cleaned.head(5)

,date,city_name,order_id,cart_id,dim_customer_key,procured_quantity,unit_selling_price,total_discount_amount,product_id,total_weighted_landing_price
0,2022-04-01,Mumbai,112246974,173273802,17995199,1,234.0,0.0,344107,202.513030
1,2022-04-01,Bengaluru,112246976,173273597,18259433,1,64.0,0.0,389676,48.714375
2,2022-04-01,Bengaluru,112247019,173123717,5402601,1,1031.0,0.0,39411,975.996000
3,2022-04-01,HR-NCR,112247045,172547459,15649744,1,57.0,0.0,369742,25.000000
4,2022-04-01,Mumbai,112247123,173081820,10127605,2,30.0,0.0,12872,57.980004


# Products

In [4]:
# Get products data

df_products = pd.read_csv('data/raw/products.csv')

In [5]:
# Basic information

df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32226 entries, 0 to 32225
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         32226 non-null  int64 
 1   product_id         32226 non-null  int64 
 2   product_name       32226 non-null  object
 3   unit               32226 non-null  object
 4   product_type       32226 non-null  object
 5   brand_name         30788 non-null  object
 6   manufacturer_name  29810 non-null  object
 7   l0_category        32226 non-null  object
 8   l1_category        32226 non-null  object
 9   l2_category        32226 non-null  object
 10  l0_category_id     32226 non-null  int64 
 11  l1_category_id     32226 non-null  int64 
 12  l2_category_id     32226 non-null  int64 
dtypes: int64(5), object(8)
memory usage: 3.2+ MB


In [6]:
# Drop "Unnamed" columns

cols_to_drop = [col for col in df_products.columns if 'Unnamed' in col]
df_products = df_products.drop(columns=cols_to_drop)


In [7]:
# Missing values

df_products.isnull().sum()

product_id              0
product_name            0
unit                    0
product_type            0
brand_name           1438
manufacturer_name    2416
l0_category             0
l1_category             0
l2_category             0
l0_category_id          0
l1_category_id          0
l2_category_id          0
dtype: int64

## Problem 04: <br> **Missing values in 'brand_name' & 'manufacturer_name'**
1. 'brand_name'         : 1,438 missing values
2. 'manufacturer_name'  : 2,416 missing values   

**Proposal:**
- Customer Segmentation and Churn Prediction -> No Problem!  
- Product Recommendation: assign 'Unknown'  

In [8]:
# Fill missing values with 'Unknown'

df_products['brand_name'] = df_products['brand_name'].fillna("Unknown")
df_products['manufacturer_name'] = df_products['manufacturer_name'].fillna("Unknown")

In [9]:
# Check the number of duplicate product_id values

duplicate_products = df_products[df_products.duplicated(subset=['product_id'], keep=False)]

print("Duplicated product_id:", duplicate_products['product_id'].nunique())
print("Total duplicated values:", len(duplicate_products))

Duplicated product_id: 0
Total duplicated values: 0


In [10]:
# Check product_name contains numeric-only values

mask = df_products["product_name"].astype(str).str.isnumeric()
numeric_names = df_products[mask]

print(f"Count: {numeric_names.shape[0]}")

Count: 0


In [11]:
# Convert ids to str

df_products = convert_id_to_str(df_products)

In [12]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32226 entries, 0 to 32225
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_id         32226 non-null  object
 1   product_name       32226 non-null  object
 2   unit               32226 non-null  object
 3   product_type       32226 non-null  object
 4   brand_name         32226 non-null  object
 5   manufacturer_name  32226 non-null  object
 6   l0_category        32226 non-null  object
 7   l1_category        32226 non-null  object
 8   l2_category        32226 non-null  object
 9   l0_category_id     32226 non-null  object
 10  l1_category_id     32226 non-null  object
 11  l2_category_id     32226 non-null  object
dtypes: object(12)
memory usage: 3.0+ MB


***Consistency check between category IDs and category names*** <br>
3 levels: l0, l1, l2

**What it does:**
1. Check if each category ID maps to only one category name.
2. Check if each category name maps to only one category ID.

In [13]:
# Consistency check between category IDs and category names

# Cross-check consistency between *_category_id and *_category
for level in ["l0", "l1", "l2"]:
    check = df_products.groupby(f"{level}_category_id")[f"{level}_category"].nunique()
    inconsistent_ids = check[check > 1]
    
    print(f"--- {level.upper()} ---")
    if inconsistent_ids.empty:
        print("[OK]: ID -> only 1 category_name")
    else:
        print("[Inconsistent]: ID → multiple category_names:")
        print(inconsistent_ids)

    check_name = df_products.groupby(f"{level}_category")[f"{level}_category_id"].nunique()
    inconsistent_names = check_name[check_name > 1]

    if inconsistent_names.empty:
        print("[OK]: category_name → only 1 ID\n")
    else:
        print("[Inconsistent]: category_name → multiple IDs:")
        print(inconsistent_names)


--- L0 ---
[OK]: ID -> only 1 category_name
[OK]: category_name → only 1 ID

--- L1 ---
[OK]: ID -> only 1 category_name
[OK]: category_name → only 1 ID

--- L2 ---
[OK]: ID -> only 1 category_name
[Inconsistent]: category_name → multiple IDs:
l2_category
Syrups    2
Name: l2_category_id, dtype: int64


**The above output shows:**
- L0 and L1 are fully consistent.
- L2 has a problem: "Syrups" is mapped to 2 different IDs.

In [14]:
# Filtered Syrups-only

syrups_detail = (
    df_products[df_products["l2_category"] == "Syrups"]
    .groupby(["l2_category", "l2_category_id"])["product_id"]
    .count()
    .reset_index()
    .rename(columns={"product_id": "count_products"})
)

print(syrups_detail)


  l2_category l2_category_id  count_products
0      Syrups           1136              43
1      Syrups           1289              31


## Problem 05: <br> **Duplicate category name with multiple IDs in L2**
Issue: The l2_category "Syrups" is linked to two different l2_category_id values <br>
-> category ID - name inconsistency. 

**Proposal:**
- l2_category_id = 1136 -> "Syrups (1136)"
- l2_category_id = 1289 -> "Syrups (1289)"


In [15]:
# export to csv

df_products_cleaned = df_products.copy()

l0_mapping = build_mapping(df_products_cleaned, "l0_category_id", "l0_category")
l1_mapping = build_mapping(df_products_cleaned, "l1_category_id", "l1_category")
l2_mapping = build_mapping(df_products_cleaned, "l2_category_id", "l2_category")

# l0_mapping.to_csv("data/supporting/l0_category.csv", index=False)
# l1_mapping.to_csv("data/supporting/l1_category.csv", index=False)
# l2_mapping.to_csv("data/supporting/l2_category.csv", index=False)

In [16]:
# test

# df_products_cleaned[df_products_cleaned['l2_category'] == 'Syrups (1136)'].head(3)
df_products_cleaned[df_products_cleaned['l2_category'] == 'Syrups (1289)'].head(3)


,product_id,product_name,unit,product_type,brand_name,manufacturer_name,l0_category,l1_category,l2_category,l0_category_id,l1_category_id,l2_category_id
30623,487147,Hershey's Genuine Chocolate Syrup - Pack of 2,2 x 1.3 kg,Syrup,Hershey's,HERSHEY INDIA PRIVATE LIMITED,Sweet Tooth,Syrups,Syrups (1289),9,1289,1289
30624,490896,Del Monte Chocolate Flavoured Syrup,600 g,Syrup,Del Monte,Del Monte Foods Pvt. Ltd,Sweet Tooth,Syrups,Syrups (1289),9,1289,1289
30640,479638,Hershey's Shell Chocolate Syrup,205 g,Syrup,Hershey's,HERSHEY INDIA PRIVATE LIMITED,Sweet Tooth,Syrups,Syrups (1289),9,1289,1289


In [29]:
# csv

# Unique product names
unique_products = df_products_cleaned[["product_id", "product_name"]].drop_duplicates()

unique_products.to_csv("data/supporting/unique_products.csv", index=False)


In [17]:
df_products_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32226 entries, 0 to 32225
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_id         32226 non-null  object
 1   product_name       32226 non-null  object
 2   unit               32226 non-null  object
 3   product_type       32226 non-null  object
 4   brand_name         32226 non-null  object
 5   manufacturer_name  32226 non-null  object
 6   l0_category        32226 non-null  object
 7   l1_category        32226 non-null  object
 8   l2_category        32226 non-null  object
 9   l0_category_id     32226 non-null  object
 10  l1_category_id     32226 non-null  object
 11  l2_category_id     32226 non-null  object
dtypes: object(12)
memory usage: 3.0+ MB


## Problem 06: <br> **1,478 *product_id* present in *Sales* but missing in *Products***



Number of *product_id* in *Sales* not found in *Products*: **1,478** <br>
-> The products metadata have not recorded <br>
**Proposal:** add dummy products to *Products*

Number of *product_id* in *Products* not found in *Sales*: **16,461** <br>
-> It can be unsold products. Totally fine!

In [27]:
# 

print("Count of product_id in *Sales*:", df_sales_cleaned["product_id"].unique().shape[0])
print("Count of product_id in *Products*:", df_products_cleaned["product_id"].unique().shape[0])
print("---")

sales_products = set(df_sales_cleaned["product_id"].unique())
products_master = set(df_products_cleaned["product_id"].unique())

missing_in_products = sales_products - products_master
print(f"[WARNING]\n Count of product_id in *Sales* not found in *Products*: {len(missing_in_products)}")
pd.Series(list(missing_in_products)).to_csv("data/supporting/missing_in_products.csv", index=False)


unused_in_sales = products_master - sales_products
print(f"[INFO]\n Count of product_id in *Products* not found in *Sales*: {len(unused_in_sales)}")


Count of product_id in *Sales*: 17243
Count of product_id in *Products*: 32226
---
[WARNING]
 Count of product_id in *Sales* not found in *Products*: 1478
[INFO]
 Count of product_id in *Products* not found in *Sales*: 16461


In [ ]:
#

# flitering
orders_with_missing_products = df_sales_cleaned[
    df_sales_cleaned["product_id"].isin(missing_in_products)
]

# to CSV
orders_with_missing_products.to_csv("data/anomalies/orders_missing_in_products.csv", index=False)

print(f"[OK] Exported {orders_with_missing_products.shape[0]} orders to 'orders_missing_in_products.csv'")

[OK] Exported 526464 orders to 'orders_missing_in_products.csv'


In [ ]:
print(orders_with_missing_products['product_id'].unique().shape[0])

1478


In [ ]:
# Statistics on the number of orders for each product_id missing in Products.

missing_product_stats = (
    orders_with_missing_products.groupby("product_id")
    .size()
    .reset_index(name="order_count")
    .sort_values(by="order_count", ascending=False)
)

missing_product_stats.to_csv(
    "data/supporting/missing_product_stats.csv", index=False
)

In [28]:
# Dummy products

df_dummy = pd.DataFrame({
    "product_id": list(missing_in_products),
    "product_name": [f"Unknown {pid}" for pid in missing_in_products],
    "unit": "Unknown",
    "product_type": "Unknown",
    "brand_name": "Unknown",
    "manufacturer_name": "Unknown",
    "l0_category": "Unknown",
    "l1_category": "Unknown",
    "l2_category": "Unknown",
    "l0_category_id": "Unknown",
    "l1_category_id": "Unknown",
    "l2_category_id": "Unknown"
})

In [37]:
df_dummy.head(5)

,product_id,product_name,l0_category,l1_category,l2_category
0,475713,Unknown 475713,Unknown,Unknown,Unknown
1,448554,Unknown 448554,Unknown,Unknown,Unknown
2,194125,Unknown 194125,Unknown,Unknown,Unknown
3,484732,Unknown 484732,Unknown,Unknown,Unknown
4,483879,Unknown 483879,Unknown,Unknown,Unknown


In [29]:
# Add dummy products to Products

df_products_cleaned_extended = pd.concat([df_products_cleaned, df_dummy])

In [34]:
df_products_cleaned_extended.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33704 entries, 0 to 1477
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_id         33704 non-null  object
 1   product_name       33704 non-null  object
 2   unit               32226 non-null  object
 3   product_type       32226 non-null  object
 4   brand_name         32226 non-null  object
 5   manufacturer_name  32226 non-null  object
 6   l0_category        33704 non-null  object
 7   l1_category        33704 non-null  object
 8   l2_category        33704 non-null  object
 9   l0_category_id     32226 non-null  object
 10  l1_category_id     32226 non-null  object
 11  l2_category_id     32226 non-null  object
dtypes: object(12)
memory usage: 3.3+ MB


In [38]:
df_products_cleaned_extended.head(5)

,product_id,product_name,unit,product_type,brand_name,manufacturer_name,l0_category,l1_category,l2_category,l0_category_id,l1_category_id,l2_category_id
0,476763,Christmas - Card,1 unit,Card,Unknown,HOT,Specials,Bill Breaker,Bill Breaker,343,1741,1741
1,483436,Plum BodyLovin' Hawaiian Rumba Shower Gel - Sa...,20 ml,Sample,Plum BodyLovin',Pureplay Skin Sciences India Pvt. Ltd.,Specials,Free Store,Free Store,343,1493,1493
2,476825,Diwali Gift Card Free - Sample,1 unit,Sample,Unknown,HOT,Specials,Bill Breaker,Bill Breaker,343,1741,1741
3,483438,Plum BodyLovin' Trippin' Mimosas Shower Gel - ...,20 ml,Sample,Plum BodyLovin',Pureplay Skin Sciences India Pvt. Ltd.,Specials,Free Store,Free Store,343,1493,1493
4,480473,Flipkart Valentine Day Greeting - Card,1 unit,Card,Flipkart,Dummy Manufacturer,Specials,Bill Breaker,Bill Breaker,343,1741,1741


# Export cleaned data to csv

In [41]:
# to csv full cleaned Sales data

df_sales_cleaned.to_csv('data/cleaned/sales_cleaned.csv', index=False)

In [30]:
# to csv full cleaned products data

df_products_cleaned_extended["product_name"] = df_products_cleaned_extended["product_name"].str.strip('"')
df_products_cleaned_extended.to_csv('data/cleaned/products_cleaned.csv', index=False)